<a href="https://colab.research.google.com/github/donaldstierman/Data-Analysis/blob/master/GoogleDataLabExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Some examples for using BigQuery from within a Jupyter notebook

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [0]:
%load_ext google.colab.data_table

In [0]:
# %unload_ext google.colab.data_table

In [4]:
# Display query output immediately

%%bigquery --project ecstatic-night-266506
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`

,total_rows
0,114420316


In [0]:
# Save output in a variable `df`

%%bigquery --project ecstatic-night-266506 df
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`

In [6]:
df

,total_rows
0,114420316


In [9]:
from google.cloud import bigquery

client = bigquery.Client(project='ecstatic-night-266506')

sample_count = 2000
row_count = client.query('''
  SELECT 
    COUNT(*) as total
  FROM `bigquery-public-data.samples.gsod`''').to_dataframe().total[0]

df = client.query('''
  SELECT
    *
  FROM
    `bigquery-public-data.samples.gsod`
  WHERE RAND() < %d/%d
''' % (sample_count, row_count)).to_dataframe()

print('Full dataset has %d rows' % row_count)

Full dataset has 114420316 rows


In [10]:
df.describe()

,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,mean_sealevel_pressure,num_mean_sealevel_pressure_samples,mean_station_pressure,num_mean_station_pressure_samples,mean_visibility,num_mean_visibility_samples,mean_wind_speed,num_mean_wind_speed_samples,max_sustained_wind_speed,max_gust_wind_speed,max_temperature,total_precipitation,snow_depth
count,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1917.000000,1917.000000,1502.000000,1502.000000,738.000000,738.000000,1784.000000,1784.000000,1976.000000,1976.000000,1941.000000,275.000000,1992.000000,1804.000000,93.000000
mean,505231.421185,90385.213855,1987.122490,6.385542,15.922189,52.061195,12.811747,41.276891,12.713615,1015.002330,11.107190,965.181030,12.098916,12.537388,12.470291,6.705668,12.757085,12.052190,25.370909,43.377962,0.072766,10.682796
std,302390.867920,26193.975533,16.494617,3.428681,8.958214,23.844631,7.865022,22.324879,7.850833,9.505118,7.452436,70.149469,7.886386,10.127723,7.744027,4.964099,7.846418,6.831360,9.555945,23.729979,0.333586,11.602011
min,10010.000000,3103.000000,1932.000000,1.000000,1.000000,-41.900002,4.000000,-47.400002,4.000000,963.700012,4.000000,629.799988,4.000000,0.000000,4.000000,0.000000,4.000000,1.000000,7.000000,-54.000000,0.000000,0.400000
25%,238692.500000,99999.000000,1977.000000,3.000000,8.000000,37.799999,7.000000,29.100000,6.000000,1009.500000,6.000000,950.400009,6.000000,6.300000,6.000000,3.400000,7.000000,7.800000,19.000000,31.250000,0.000000,3.100000
50%,540195.000000,99999.000000,1989.000000,6.000000,16.000000,55.000000,8.000000,43.900002,8.000000,1014.900024,8.000000,994.000000,8.000000,9.700000,8.000000,5.600000,8.000000,10.100000,23.299999,46.400002,0.000000,7.500000
75%,725115.250000,99999.000000,2001.000000,9.000000,24.000000,70.599998,23.000000,56.799999,23.000000,1020.099976,18.000000,1008.875015,23.000000,15.100000,23.000000,8.700000,23.000000,15.500000,29.500000,60.799999,0.010000,15.000000
max,999999.000000,99999.000000,2010.000000,12.000000,31.000000,105.000000,24.000000,84.199997,24.000000,1062.300049,24.000000,1034.500000,24.000000,99.400002,24.000000,38.500000,24.000000,55.900002,77.900002,88.199997,6.420000,63.000000


In [11]:
df.head(10)

,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,mean_sealevel_pressure,num_mean_sealevel_pressure_samples,mean_station_pressure,num_mean_station_pressure_samples,mean_visibility,num_mean_visibility_samples,mean_wind_speed,num_mean_wind_speed_samples,max_sustained_wind_speed,max_gust_wind_speed,max_temperature,max_temperature_explicit,min_temperature,min_temperature_explicit,total_precipitation,snow_depth,fog,rain,snow,hail,thunder,tornado
0,416200,99999,1981,12,29,46.900002,7,25.100000,7.0,NaN,NaN,NaN,NaN,NaN,NaN,0.4,7.0,2.9,NaN,37.400002,True,None,None,NaN,NaN,False,False,False,False,False,False
1,719675,99999,1985,10,9,28.400000,6,25.400000,6.0,NaN,NaN,NaN,NaN,14.900000,6.0,1.3,6.0,4.1,NaN,23.000000,True,None,None,0.00,NaN,False,False,False,False,False,False
2,723417,93988,1996,10,22,60.000000,24,56.799999,24.0,NaN,NaN,NaN,NaN,7.000000,24.0,10.1,24.0,21.0,28.0,48.200001,True,None,None,3.55,NaN,False,False,False,False,False,False
3,135980,99999,2001,5,16,59.700001,7,45.200001,7.0,NaN,NaN,NaN,NaN,12.400000,7.0,1.9,7.0,5.8,NaN,45.500000,False,None,None,0.00,NaN,False,False,False,False,False,False
4,574510,99999,1984,7,5,61.500000,4,61.400002,4.0,NaN,NaN,816.400024,4.0,0.000000,4.0,11.7,4.0,15.5,NaN,58.099998,False,None,None,0.28,NaN,False,False,False,False,False,False
5,605600,99999,2008,8,29,82.400002,5,53.000000,5.0,NaN,NaN,895.799988,5.0,12.400000,5.0,1.2,5.0,6.0,NaN,63.099998,False,None,None,0.08,NaN,False,False,False,False,False,False
6,943670,99999,1960,7,4,60.500000,7,49.000000,7.0,NaN,NaN,1017.200012,7.0,NaN,NaN,12.5,7.0,22.9,NaN,50.200001,True,None,None,0.00,NaN,False,False,False,False,False,False
7,945420,99999,1999,10,20,64.900002,4,46.599998,4.0,1016.5,4.0,976.500000,4.0,24.900000,4.0,3.3,4.0,7.0,NaN,53.200001,True,None,None,0.00,NaN,False,False,False,False,False,False
8,536980,99999,1979,6,9,75.500000,7,41.299999,7.0,1008.5,7.0,NaN,NaN,18.200001,7.0,5.0,7.0,7.8,NaN,62.599998,False,None,None,0.00,NaN,False,False,False,False,False,False
9,644010,99999,1981,3,24,79.599998,7,71.900002,7.0,1010.0,7.0,NaN,NaN,12.000000,7.0,1.9,7.0,3.9,NaN,73.400002,False,None,None,0.00,NaN,False,False,False,False,False,False


In [12]:
# 10 highest total_precipitation samples
df.sort_values('total_precipitation', ascending=False).head(10)[['station_number', 'year', 'month', 'day', 'total_precipitation']]

,station_number,year,month,day,total_precipitation
847,156010,2003,6,25,6.42
1806,76030,1988,6,17,4.72
571,976900,1996,2,22,4.13
1325,822810,1991,2,19,4.13
1090,587370,1980,4,29,4.13
2,723417,1996,10,22,3.55
441,386130,1961,2,4,2.95
1710,549450,1989,8,25,1.93
212,691490,1999,2,22,1.88
1610,117100,2007,7,21,1.82


In [14]:
import pandas as pd

sample_count = 2000
df = pd.io.gbq.read_gbq('''
  SELECT name, SUM(number) as count
  FROM `bigquery-public-data.usa_names.usa_1910_2013`
  WHERE state = 'TX'
  GROUP BY name
  ORDER BY count DESC
  LIMIT 100
''', project_id='ecstatic-night-266506', dialect='standard')

df.head()

,name,count
0,James,272793
1,John,235139
2,Michael,225320
3,Robert,220399
4,David,219028
